# Tutorial: FDDBenchmark

Fault detection and diagnosis (FDD) tasks are highly important in process monitoring to prevent failures and reduce equipment downtime, for example, in the chemical industry.

* **Fault detection** is used to define whether a fault has occurred​ (binary classification).
* **Fault diagnosis** aims to determine types of faults​ (milticlass classification).

Fault detection and diagnosis (FDD) benchmark aims to make processes of training and testing ML models simple and clear, as well as fast and cheap in memory. The benchmark consists of 3 objects: `FDDDataset`, `FDDDataloader`, `FDDEvaluator`.

<img src='https://github.com/airi-industrial-ai/fddbenchmark/raw/main/tutorial/fddbench_overview.png' width=600>

This tutorial describes an example of training and testing procedures, then proposes short templates for external usage.

### Part 1. Training procedure

In [25]:
!pip install git+https://github.com/airi-industrial-ai/fddbenchmark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/airi-industrial-ai/fddbenchmark to /tmp/pip-req-build-uxx20xyt
  Running command git clone --filter=blob:none --quiet https://github.com/airi-industrial-ai/fddbenchmark /tmp/pip-req-build-uxx20xyt
  Resolved https://github.com/airi-industrial-ai/fddbenchmark to commit 1e5b5fa4692d09d5f2c660457558329c2d19c094
  Preparing metadata (setup.py) ... done


In [26]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from fddbenchmark import FDDDataset, FDDDataloader, FDDEvaluator

Properties of FDDDataset:
* `df` — pandas dataframe, timeseries with indices: `run_id` defines a run of the process, `step` defines a sequence of rows, the larger the later
* `labels` — pandas series, a class of faults, 0 is the normal behaviour, the same indices as in `df`
* `train_mask` — pandas series, a mask of train data
* `test_mask` — pandas series, a mask of test data

The `small_tep` dataset is based on [Additional Tennessee Eastman Process Simulation Data](https://doi.org/10.7910/DVN/6C3JR1). The dataset contains only a few simulation runs for each fault type.

In [27]:
small_tep = FDDDataset(name='small_tep')
small_tep.df.head()

Reading data/small_tep/test_mask.csv: 100%|██████████| 153300/153300 [00:00<00:00, 2453174.32it/s]


xmeas_1  xmeas_2  xmeas_3  xmeas_4  xmeas_5  xmeas_6  \
run_id    sample                                                         
413402073 1       0.25038   3674.0   4529.0   9.2320   26.889   42.402   
          2       0.25109   3659.4   4556.6   9.4264   26.721   42.576   
          3       0.25038   3660.3   4477.8   9.4426   26.875   42.070   
          4       0.24977   3661.3   4512.1   9.4776   26.758   42.063   
          5       0.29405   3679.0   4497.0   9.3381   26.889   42.650   

                  xmeas_7  xmeas_8  xmeas_9  xmeas_10  ...   xmv_2   xmv_3  \
run_id    sample                                       ...                   
413402073 1        2704.3   74.863   120.41   0.33818  ...  53.744  24.657   
          2        2705.0   75.000   120.41   0.33620  ...  53.414  24.588   
          3        2706.2   74.771   120.42   0.33563  ...  54.357  24.666   
          4        2707.2   75.224   120.39   0.33553  ...  53.946  24.725   
          5        2705.1   75.388   120.39   0.32632  ...  53.658  28.797   

                   xmv_4   xmv_5   xmv_6   xmv_7   xmv_8   xmv_9  xmv_10  \
run_id    sample                                                           
413402073 1       62.544  22.137  39.935  42.323  47.757  47.510  41.258   
          2       59.259  22.084  40.176  38.554  43.692  47.427  41.359   
          3       61.275  22.380  40.244  38.990  46.699  47.468  41.199   
          4       59.856  22.277  40.257  38.072  47.541  47.658  41.643   
          5       60.717  21.947  39.144  41.955  47.645  47.346  41.507   

                  xmv_11  
run_id    sample          
413402073 1       18.447  
          2       17.194  
          3       20.530  
          4       18.089  
          5       18.461  

[5 rows x 52 columns]

In [28]:
small_tep.labels.head()

run_id     sample
413402073  1         0
           2         0
           3         0
           4         0
           5         0
Name: labels, dtype: int64

`FDDDataloader` is an iterator that dynamically slices a dataset into batches of samples of the fixed length (window size) using the fixed step size. For example:

<img src='https://github.com/airi-industrial-ai/fddbenchmark/raw/main/tutorial/window_step_size.png' width=1000>

Labels are not sliced, instead they are reduced: if at least a single time stamp in a sample is fault then the entire sample is fault. The iterator consists of tuples:
* `ts` — numpy array, samples of the shape "batch size" X "window size" X "time series dimensionality"
* `index` — pandas index, defines the ordering of samples in `ts`
* `label` — pandas series, label of samples

Let us create a dataloader that iterates over train set. The dataloader allows to iterate by mini-batches and also by a single batch. We set `minibatch_training` is false, then the dataloader contains a single batch and we can break a loop after the first iteration.

In [29]:
train_dl = FDDDataloader(
    dataframe=small_tep.df,
    labels=small_tep.labels,
    mask=small_tep.train_mask,
    window_size=10,
    step_size=5,
    minibatch_training=False,
)

for train_ts, train_index, train_label in train_dl:
    break
train_ts.shape, train_index.shape, train_label.shape

Creating sequence of samples: 100%|██████████| 105/105 [00:00<00:00, 647.97it/s]


((10290, 10, 52), (10290,), (10290,))

In this setting, each sample is a matrix of the shape "window size" X "time series dimensionality", but we will train the model that takes a vectors, not matrices. Let us reshape samples to the vector form.

In [30]:
v_train_ts = train_ts.reshape(train_ts.shape[0], -1)
v_train_ts.shape

(10290, 520)

Train a simple model based on PCA and gradient boosting classifier. We reweight samples to balance normal and faulty classes.

In [31]:
model = make_pipeline(
    StandardScaler(), 
    PCA(n_components=32), 
    GradientBoostingClassifier(n_estimators=100, max_depth=3, verbose=1)
)
weight = np.ones(train_label.shape[0])
weight[train_label != 0] = (train_label == 0).sum() / (train_label != 0).sum()
model.fit(
    v_train_ts, 
    train_label, 
    gradientboostingclassifier__sample_weight=weight
)

      Iter       Train Loss   Remaining Time 
         1           1.6199            4.94m
         2           1.5244            5.57m
         3           1.4556            5.56m
         4           1.3985            6.16m
         5           1.3491            6.96m
         6           1.3078            7.06m
         7           1.2698            6.67m
         8           1.2359            6.42m
         9           1.2057            6.12m
        10           1.1799            5.87m
        20           0.9978            4.62m
        30           0.8886            3.86m
        40           0.8207            3.27m
        50           0.7600            2.79m
        60           0.7152            2.20m
        70           0.6770            1.63m
        80           0.6445            1.07m
        90           0.6149           32.03s
       100           0.5862            0.00s


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=32)),
                ('gradientboostingclassifier',
                 GradientBoostingClassifier(verbose=1))])

### Part 2. Testing procedure

Create a dataloader using the test mask for testing.

In [32]:
test_dl = FDDDataloader(
    dataframe=small_tep.df,
    labels=small_tep.labels,
    mask=small_tep.test_mask,
    window_size=10, 
    step_size=5, 
)
for test_ts, test_index, test_label in test_dl:
    break
test_ts.shape, test_index.shape, test_label.shape

Creating sequence of samples: 100%|██████████| 105/105 [00:00<00:00, 573.93it/s]


((19950, 10, 52), (19950,), (19950,))

Generate predictions and index them to define an ordering of rows. It is important to calculate detection delay metrics.

In [33]:
v_test_ts = test_ts.reshape(test_ts.shape[0], -1)
preds = model.predict(v_test_ts)
preds = pd.Series(preds, index=test_index, name='labels', dtype=int)

Evaluate the model using `FDDEvaluator`. `evaluate` method returns a dictionary of metrics, while `print_metrics` print them.

In [34]:
evaluator = FDDEvaluator(
    step_size=test_dl.step_size
)
evaluator.print_metrics(test_label, preds)

FDD metrics
-----------------
TPR/FPR:
    Fault 01: 0.9648/0.0002
    Fault 02: 0.9761/0.0005
    Fault 03: 0.0075/0.0010
    Fault 04: 0.6000/0.0025
    Fault 05: 0.8113/0.0180
    Fault 06: 0.9774/0.0000
    Fault 07: 0.9736/0.0000
    Fault 08: 0.7296/0.0054
    Fault 09: 0.0013/0.0111
    Fault 10: 0.1648/0.0067
    Fault 11: 0.0440/0.0007
    Fault 12: 0.5069/0.0052
    Fault 13: 0.7220/0.0054
    Fault 14: 0.9874/0.0000
    Fault 15: 0.0101/0.0121
    Fault 16: 0.0164/0.0002
    Fault 17: 0.7774/0.0104
    Fault 18: 0.8428/0.0002
    Fault 19: 0.0088/0.0010
    Fault 20: 0.3258/0.0020
Detection TPR: 0.6455
Detection FPR: 0.0827
Average Detection Delay (ADD): 169.25
Total Correct Diagnosis Rate (Total CDR): 0.8092

Clustering metrics
-----------------
Adjusted Rand Index (ARI): 0.2850
Normalized Mutual Information (NMI): 0.5907
Unsupervised Clustering Accuracy (ACC): 0.6034


### Part 3. Template for single-batch training and testing

Here is the shortest possible template for single-batch training and testing a model. Copy-paste and modify.

```python
small_tep = FDDDataset(name='small_tep')
train_dl = FDDDataloader(
    dataframe=small_tep.df,
    labels=small_tep.labels,
    mask=small_tep.train_mask,
    window_size=10,
    step_size=5,
    minibatch_training=False,
)
for train_ts, train_index, train_label in train_dl:
    break
# model = ... define and train your model
test_dl = FDDDataloader(
    dataframe=small_tep.df,
    labels=small_tep.labels,
    mask=small_tep.test_mask,
    window_size=10, 
    step_size=5,
    minibatch_training=False
)
for test_ts, test_index, test_label in test_dl:
    break
# preds = ... generate predictions
preds = pd.Series(pred, index=test_index, dtype=int)
evaluator = FDDEvaluator(
    step_size=test_dl.step_size
)
metrics = evaluator.evaluate(test_label, preds)
# store metrics
```

### Part 4. Template for mini-batch training and testing

Here is the shortest possible template for mini-batch training and testing a model. Copy-paste and modify.

```python
small_tep = FDDDataset(name='small_tep')
train_dl = FDDDataloader(
    dataframe=small_tep.df,
    labels=small_tep.labels,
    mask=small_tep.train_mask,
    window_size=10,
    step_size=5,
    minibatch_training=True,
    batch_size=128,
    shuffle=True
)
# model = ... define your model
for train_ts, train_index, train_label in train_dl:
    # train your model
test_dl = FDDDataloader(
    dataframe=small_tep.df,
    labels=small_tep.labels,
    mask=small_tep.test_mask,
    window_size=10, 
    step_size=5, 
    minibatch_training=True,
    batch_size=128,
)
preds = []
labels = []
for test_ts, test_index, test_label in test_dl:
    # pred = ... generate predictions
    preds.append(pd.Series(pred, index=test_index, dtype=int))
    labels.append(test_label)
preds = pd.concat(preds)
test_label = pd.concat(labels)
evaluator = FDDEvaluator(
    step_size=test_dl.step_size
)
metrics = evaluator.evaluate(test_labels, preds)
# store metrics
```